In [27]:
file = "https://github.com/AndrewKJacobs2/Coursera_Capstone/blob/master/US%20Latitudes%20and%20Longitudes%20NEW.xlsx?raw=true"
file

'https://github.com/AndrewKJacobs2/Coursera_Capstone/blob/master/US%20Latitudes%20and%20Longitudes%20NEW.xlsx?raw=true'

In [28]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [29]:
citydata = pd.read_excel(file)

In [30]:
latitude = 41
longitude = -93

map_uscanada = folium.Map(location=[latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, label in zip(citydata['Latitude'], citydata['Longitude'], citydata['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uscanada)  
    
map_uscanada

In [31]:
CLIENT_ID = 'EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH' # your Foursquare ID
CLIENT_SECRET = '1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH
CLIENT_SECRET:1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP


In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    42.652500, 
    -73.757220, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EGJ51MQNA4AD1YEPXGHZCK4VFATFLCK2BR0XSRNTYAR2VYPH&client_secret=1KCUZYJJVTZO2MS4SHCV5SYKWJBYVNQOYD325X5W2NP51GEP&v=20180605&ll=42.6525,-73.75722&radius=10000&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2e5ec049b14d0023abd8a0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Albany',
  'headerFullLocation': 'Albany',
  'headerLocationGranularity': 'city',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 42.74250009000009,
    'lng': -73.63507871822407},
   'sw': {'lat': 42.562499909999914, 'lng': -73.87936128177594}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55f6d343498edb711f07a4e1',
       'name': 'Renaissance Albany Hotel',
       'location': {'address': '144 State St',
        'crossStreet': 'Eagle Street',
        'lat': 42.65062486985366,
        'lng': -73.7556865811348,
       

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Renaissance Albany Hotel,Hotel,42.650625,-73.755687
1,Iron Gate Cafe,Café,42.655974,-73.762504
2,City Beer Hall,Pub,42.649660,-73.754787
3,The Olde English Pub & Pantry,Pub,42.653958,-73.748563
4,New York State Museum,Museum,42.648974,-73.761258


In [36]:
nearby_venues.tail()

,name,categories,lat,lng
95,Greenbush YMCA,Gym / Fitness Center,42.616988,-73.695576
96,Jim's Tastee-Freez,Ice Cream Shop,42.630996,-73.805044
97,Trader Joe's,Grocery Store,42.714444,-73.812380
98,Perfect Blend,Coffee Shop,42.622032,-73.832952
99,Bruegger's,Bagel Shop,42.628614,-73.812358


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
uscanven = getNearbyVenues(names=citydata['City'],
                                   latitudes=citydata['Latitude'],
                                   longitudes=citydata['Longitude']
                                  )

Albany, N.Y.
Albuquerque, N.M.
Amarillo, Tex.
Anchorage, Alaska
Atlanta, Ga.
Austin, Tex.
Baker, Ore.
Baltimore, Md.
Bangor, Maine
Birmingham, Ala.
Bismarck, N.D.
Boise, Idaho
Boston, Mass.
Buffalo, N.Y.
Calgary, Alba., Can.
Carlsbad, N.M.
Charleston, S.C.
Charleston, W. Va.
Charlotte, N.C.
Cheyenne, Wyo.
Chicago, Ill.
Cincinnati, Ohio
Cleveland, Ohio
Columbia, S.C.
Columbus, Ohio
Dallas, Tex.
Denver, Colo.
Des Moines, Iowa
Detroit, Mich.
Dubuque, Iowa
Duluth, Minn.
Eastport, Maine
Edmonton, Alb., Can.
El Centro, Calif.
El Paso, Tex.
Eugene, Ore.
Fargo, N.D.
Flagstaff, Ariz.
Fort Worth, Tex.
Fresno, Calif.
Grand Junction, Colo.
Grand Rapids, Mich.
Havre, Mont.
Helena, Mont.
Honolulu, Hawaii
Hot Springs, Ark.
Houston, Tex.
Idaho Falls, Idaho
Indianapolis, Ind.
Jackson, Miss.
Jacksonville, Fla.
Juneau, Alaska
Kansas City, Mo.
Key West, Fla.
Kingston, Ont., Can.
Klamath Falls, Ore.
Knoxville, Tenn.
Las Vegas, Nev.
Lewiston, Idaho
Lincoln, Neb.
Long Beach, Calif.
Los Angeles, Calif.
Louisv

In [43]:
uscanven

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Albany, N.Y.",42.65250,-73.75722,Renaissance Albany Hotel,42.650625,-73.755687,Hotel
1,"Albany, N.Y.",42.65250,-73.75722,City Beer Hall,42.649660,-73.754787,Pub
2,"Albany, N.Y.",42.65250,-73.75722,The Egg,42.650593,-73.758390,Performing Arts Venue
3,"Albany, N.Y.",42.65250,-73.75722,Iron Gate Cafe,42.655974,-73.762504,Café
4,"Albany, N.Y.",42.65250,-73.75722,Public House 42,42.650171,-73.756492,English Restaurant
5,"Albany, N.Y.",42.65250,-73.75722,New York State Museum,42.648974,-73.761258,Museum
6,"Albany, N.Y.",42.65250,-73.75722,Cider Belly Donuts,42.650487,-73.752350,Café
7,"Albany, N.Y.",42.65250,-73.75722,Empire State Plaza,42.650492,-73.759504,Plaza
8,"Albany, N.Y.",42.65250,-73.75722,Palace Theatre,42.654736,-73.750192,Theater
9,"Albany, N.Y.",42.65250,-73.75722,Stacks Espresso Bar - Downtown,42.650257,-73.750645,Café


In [44]:
# one hot encoding
uscan_onehot = pd.get_dummies(uscanven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
uscan_onehot['City'] = uscanven['City'] 

# move neighborhood column to the first column
fixed_columns = [uscan_onehot.columns[-1]] + list(uscan_onehot.columns[:-1])
uscan_onehot = uscan_onehot[fixed_columns]

uscan_onehot.head()

,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Albany, N.Y.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
uscangrouped = uscan_onehot.groupby('City').mean().reset_index()
uscangrouped

,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Albany, N.Y.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.010000,0.01,0.00,0.000000,0.000000,0.010000,0.000000
1,"Albuquerque, N.M.",0.010000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,"Amarillo, Tex.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.015385,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,"Anchorage, Alaska",0.000000,0.020000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,"Atlanta, Ga.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.000000
5,"Austin, Tex.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.020000,0.000000
6,"Baker, Ore.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.025000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
7,"Baltimore, Md.",0.000000,0.000000,0.0,0.00,0.0,0.01,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,"Bangor, Maine",0.000000,0.000000,0.0,0.00,0.0,0.00,0.016667,0.033333,0.05,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
9,"Birmingham, Ala.",0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,...,0.0,0.00,0.000000,0.000000,0.00,0.00,0.016129,0.000000,0.000000,0.000000


In [46]:
uscangrouped.shape

(126, 431)

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = uscangrouped['City']

for ind in np.arange(uscangrouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(uscangrouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Albany, N.Y.",Pub,Café,Coffee Shop,Pizza Place,Theater,New American Restaurant,Brewery,Convenience Store,American Restaurant,...,Pharmacy,Latin American Restaurant,Hotel,Mexican Restaurant,Asian Restaurant,Noodle House,Beer Bar,Steakhouse,Food & Drink Shop,Donut Shop
1,"Albuquerque, N.M.",Hotel,Mexican Restaurant,Furniture / Home Store,Burger Joint,Shoe Store,Brewery,Sandwich Place,Fast Food Restaurant,Grocery Store,...,Pizza Place,Rental Car Location,Thrift / Vintage Store,Gym,Park,Restaurant,Diner,Italian Restaurant,Video Store,Gas Station
2,"Amarillo, Tex.",Sandwich Place,Restaurant,American Restaurant,Bank,Gas Station,Bar,Park,Pizza Place,Sushi Restaurant,...,Burrito Place,Bakery,Baseball Stadium,Grocery Store,Gift Shop,Gay Bar,Theater,Hotel,Liquor Store,Coffee Shop
3,"Anchorage, Alaska",Seafood Restaurant,Park,Coffee Shop,Clothing Store,Bar,Steakhouse,Sporting Goods Shop,Cosmetics Shop,Pizza Place,...,Restaurant,Asian Restaurant,Mexican Restaurant,Breakfast Spot,Pub,Department Store,Sushi Restaurant,Thai Restaurant,Bakery,Brewery
4,"Atlanta, Ga.",Hotel,American Restaurant,Aquarium,Coffee Shop,Hotel Bar,Sandwich Place,Museum,Mediterranean Restaurant,Lounge,...,Burger Joint,Fast Food Restaurant,Seafood Restaurant,Caribbean Restaurant,Gym,Music Venue,Pizza Place,Park,Dive Bar,Burrito Place


In [49]:
# set number of clusters
kclusters = 10

uscangrouped_clustering = uscangrouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uscangrouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 7, 4, 2, 1, 2, 2, 7, 0], dtype=int32)

In [50]:
kmeans.labels_[0:10].dtype

dtype('int32')

In [51]:
# add clustering labels
city_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_, int)

uscan_merged = citydata

newheaders = ('City', 'Latitude', 'Longitude')
uscan_merged.columns=newheaders

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
uscan_merged = uscan_merged.join(city_venues_sorted.set_index('City'), on='City')

uscan_merged.head() # check the last columns!

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Albany, N.Y.",42.65250,-73.75722,4,Pub,Café,Coffee Shop,Pizza Place,Theater,New American Restaurant,...,Pharmacy,Latin American Restaurant,Hotel,Mexican Restaurant,Asian Restaurant,Noodle House,Beer Bar,Steakhouse,Food & Drink Shop,Donut Shop
1,"Albuquerque, N.M.",35.11083,-106.61000,2,Hotel,Mexican Restaurant,Furniture / Home Store,Burger Joint,Shoe Store,Brewery,...,Pizza Place,Rental Car Location,Thrift / Vintage Store,Gym,Park,Restaurant,Diner,Italian Restaurant,Video Store,Gas Station
2,"Amarillo, Tex.",35.19917,-101.84528,7,Sandwich Place,Restaurant,American Restaurant,Bank,Gas Station,Bar,...,Burrito Place,Bakery,Baseball Stadium,Grocery Store,Gift Shop,Gay Bar,Theater,Hotel,Liquor Store,Coffee Shop
3,"Anchorage, Alaska",61.21700,-149.90000,4,Seafood Restaurant,Park,Coffee Shop,Clothing Store,Bar,Steakhouse,...,Restaurant,Asian Restaurant,Mexican Restaurant,Breakfast Spot,Pub,Department Store,Sushi Restaurant,Thai Restaurant,Bakery,Brewery
4,"Atlanta, Ga.",33.75500,-84.39000,2,Hotel,American Restaurant,Aquarium,Coffee Shop,Hotel Bar,Sandwich Place,...,Burger Joint,Fast Food Restaurant,Seafood Restaurant,Caribbean Restaurant,Gym,Music Venue,Pizza Place,Park,Dive Bar,Burrito Place


In [52]:
uscan_merged

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Albany, N.Y.",42.652500,-73.757220,4,Pub,Café,Coffee Shop,Pizza Place,Theater,New American Restaurant,...,Pharmacy,Latin American Restaurant,Hotel,Mexican Restaurant,Asian Restaurant,Noodle House,Beer Bar,Steakhouse,Food & Drink Shop,Donut Shop
1,"Albuquerque, N.M.",35.110830,-106.610000,2,Hotel,Mexican Restaurant,Furniture / Home Store,Burger Joint,Shoe Store,Brewery,...,Pizza Place,Rental Car Location,Thrift / Vintage Store,Gym,Park,Restaurant,Diner,Italian Restaurant,Video Store,Gas Station
2,"Amarillo, Tex.",35.199170,-101.845280,7,Sandwich Place,Restaurant,American Restaurant,Bank,Gas Station,Bar,...,Burrito Place,Bakery,Baseball Stadium,Grocery Store,Gift Shop,Gay Bar,Theater,Hotel,Liquor Store,Coffee Shop
3,"Anchorage, Alaska",61.217000,-149.900000,4,Seafood Restaurant,Park,Coffee Shop,Clothing Store,Bar,Steakhouse,...,Restaurant,Asian Restaurant,Mexican Restaurant,Breakfast Spot,Pub,Department Store,Sushi Restaurant,Thai Restaurant,Bakery,Brewery
4,"Atlanta, Ga.",33.755000,-84.390000,2,Hotel,American Restaurant,Aquarium,Coffee Shop,Hotel Bar,Sandwich Place,...,Burger Joint,Fast Food Restaurant,Seafood Restaurant,Caribbean Restaurant,Gym,Music Venue,Pizza Place,Park,Dive Bar,Burrito Place
5,"Austin, Tex.",30.267000,-97.733000,1,Food Truck,Hotel,Coffee Shop,Bar,Pizza Place,Dive Bar,...,Taco Place,Lounge,Juice Bar,New American Restaurant,Capitol Building,Burger Joint,Music Venue,Movie Theater,Ice Cream Shop,American Restaurant
6,"Baker, Ore.",44.775000,-117.834440,2,Coffee Shop,Hotel,Pizza Place,Fast Food Restaurant,Gas Station,Grocery Store,...,Diner,Sandwich Place,Tennis Court,Bowling Alley,Mexican Restaurant,Automotive Shop,Motel,Discount Store,Restaurant,Pharmacy
7,"Baltimore, Md.",39.283000,-76.617000,2,Hotel,American Restaurant,Aquarium,Seafood Restaurant,Bar,Sushi Restaurant,...,Dessert Shop,Gym,Burger Joint,Steakhouse,Mexican Restaurant,Middle Eastern Restaurant,Park,Peruvian Restaurant,Italian Restaurant,Coffee Shop
8,"Bangor, Maine",44.800000,-68.800000,7,Rental Car Location,Pizza Place,Hotel,Airport Terminal,American Restaurant,Coffee Shop,...,Gas Station,Pharmacy,Park,Golf Course,Baseball Stadium,Grocery Store,Soccer Field,Ice Cream Shop,Sushi Restaurant,Gift Shop
9,"Birmingham, Ala.",33.653330,-86.808890,0,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Gas Station,Pharmacy,Discount Store,...,Pet Store,Sushi Restaurant,Big Box Store,Martial Arts Dojo,Photography Lab,Breakfast Spot,Burrito Place,Smoke Shop,Dance Studio,Fabric Shop


In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(uscan_merged['Latitude'], uscan_merged['Longitude'], uscan_merged['City'], uscan_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 9]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
79,"Nome, Alaska",64.50389,-165.39944,9,Hotel,Grocery Store,Bakery,Restaurant,Tourist Information Center,Sandwich Place,...,Drive-in Theater,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Zoo,Donut Shop,Fabric Shop


In [60]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 0]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
9,"Birmingham, Ala.",33.653330,-86.808890,0,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Gas Station,Pharmacy,Discount Store,...,Pet Store,Sushi Restaurant,Big Box Store,Martial Arts Dojo,Photography Lab,Breakfast Spot,Burrito Place,Smoke Shop,Dance Studio,Fabric Shop
15,"Carlsbad, N.M.",32.411940,-104.236390,0,Pizza Place,Mexican Restaurant,Hotel,Burger Joint,Convenience Store,Fast Food Restaurant,...,Coffee Shop,Movie Theater,Grocery Store,Asian Restaurant,Gas Station,Breakfast Spot,Café,Pharmacy,Gym / Fitness Center,Discount Store
19,"Cheyenne, Wyo.",41.145560,-104.801940,0,Fast Food Restaurant,Mexican Restaurant,Clothing Store,Sandwich Place,Pizza Place,Department Store,...,Asian Restaurant,Gas Station,Movie Theater,Gym / Fitness Center,Video Game Store,Brewery,Chinese Restaurant,Lingerie Store,Coffee Shop,Steakhouse
20,"Chicago, Ill.",41.836940,-87.684720,0,Mexican Restaurant,Sandwich Place,Taco Place,Italian Restaurant,Supermarket,Hot Dog Joint,...,Video Store,Fried Chicken Joint,Bank,Cosmetics Shop,Thrift / Vintage Store,Mobile Phone Shop,Train Station,Ice Cream Shop,Pharmacy,Discount Store
33,"El Centro, Calif.",32.800000,-115.567000,0,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,...,Bank,Restaurant,Discount Store,Convenience Store,Pharmacy,Sporting Goods Shop,Supermarket,Fried Chicken Joint,Steakhouse,Pet Store
39,"Fresno, Calif.",36.750000,-119.767000,0,Mexican Restaurant,Grocery Store,Taco Place,Coffee Shop,Sandwich Place,Soup Place,...,Mediterranean Restaurant,Fried Chicken Joint,Train Station,Mobile Phone Shop,Convenience Store,Breakfast Spot,Discount Store,Bakery,Burger Joint,Paper / Office Supplies Store
40,"Grand Junction, Colo.",39.067000,-108.567000,0,Mexican Restaurant,Pizza Place,Coffee Shop,Fast Food Restaurant,Salon / Barbershop,Hotel,...,Shipping Store,Bagel Shop,Breakfast Spot,Pub,Diner,Dessert Shop,Paper / Office Supplies Store,Park,Convenience Store,American Restaurant
42,"Havre, Mont.",48.550000,-109.683000,0,Fast Food Restaurant,Food,Pizza Place,Sandwich Place,Steakhouse,Business Service,...,Diner,ATM,Hotel,American Restaurant,Irish Pub,Inn,Cycle Studio,Creperie,Ethiopian Restaurant,Event Space
43,"Helena, Mont.",46.595806,-112.027031,0,American Restaurant,Fast Food Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Mexican Restaurant,...,Pharmacy,Gift Shop,Bar,Bank,Park,Motel,Sporting Goods Shop,Italian Restaurant,Gym / Fitness Center,Mobile Phone Shop
45,"Hot Springs, Ark.",34.497220,-93.055280,0,Fast Food Restaurant,Hotel,Pizza Place,Mexican Restaurant,BBQ Joint,Bath House,...,Seafood Restaurant,Burger Joint,Café,Diner,ATM,Candy Store,Sandwich Place,Brewery,Scenic Lookout,Pharmacy


In [61]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 1]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,"Austin, Tex.",30.267000,-97.733000,1,Food Truck,Hotel,Coffee Shop,Bar,Pizza Place,Dive Bar,...,Taco Place,Lounge,Juice Bar,New American Restaurant,Capitol Building,Burger Joint,Music Venue,Movie Theater,Ice Cream Shop,American Restaurant
21,"Cincinnati, Ohio",39.100000,-84.517000,1,Bar,American Restaurant,Hotel,Sandwich Place,Mexican Restaurant,Coffee Shop,...,Park,Steakhouse,Café,Cocktail Bar,Pizza Place,Bakery,New American Restaurant,Performing Arts Venue,Ice Cream Shop,Gym
22,"Cleveland, Ohio",41.482220,-81.669720,1,Pub,Bar,Lounge,Sushi Restaurant,Café,Coffee Shop,...,Southern / Soul Food Restaurant,Snack Place,Cajun / Creole Restaurant,Sandwich Place,Cocktail Bar,College Basketball Court,Pizza Place,Thai Restaurant,Diner,Park
24,"Columbus, Ohio",39.983000,-82.983000,1,Bar,Pizza Place,Café,American Restaurant,Cocktail Bar,Sandwich Place,...,Nightclub,Boutique,Coffee Shop,Arts & Crafts Store,Dive Bar,Mexican Restaurant,Museum,Salon / Barbershop,Wine Bar,Fast Food Restaurant
34,"El Paso, Tex.",31.759206,-106.490175,1,Bar,Coffee Shop,Mexican Restaurant,Fast Food Restaurant,Lounge,Hotel,...,Discount Store,Bistro,Pharmacy,History Museum,Cocktail Bar,Fried Chicken Joint,Burger Joint,Brewery,Restaurant,College Bookstore
36,"Fargo, N.D.",46.877220,-96.789440,1,Coffee Shop,Bar,American Restaurant,Brewery,Theater,Burger Joint,...,Gift Shop,Diner,Juice Bar,Grocery Store,Lounge,Boutique,Taco Place,Cocktail Bar,Sandwich Place,Deli / Bodega
37,"Flagstaff, Ariz.",35.199170,-111.631110,1,Coffee Shop,Brewery,American Restaurant,Mexican Restaurant,Bar,Pizza Place,...,Grocery Store,Hot Dog Joint,Breakfast Spot,Hotel,Sporting Goods Shop,Sandwich Place,Cocktail Bar,Japanese Restaurant,Fast Food Restaurant,Burger Joint
50,"Jacksonville, Fla.",30.336940,-81.661390,1,Sandwich Place,Bar,Coffee Shop,Brewery,Hotel,Park,...,Food Truck,Theater,Steakhouse,Lounge,Art Gallery,Shipping Store,Burrito Place,Rental Car Location,Convenience Store,Caribbean Restaurant
51,"Juneau, Alaska",58.300323,-134.417639,1,Seafood Restaurant,Coffee Shop,Bar,Gift Shop,Café,Pizza Place,...,Cruise,Mexican Restaurant,Hotel,Jewelry Store,Movie Theater,Scenic Lookout,Grocery Store,Pier,Breakfast Spot,Asian Restaurant
56,"Knoxville, Tenn.",35.961700,-83.923200,1,Bar,American Restaurant,Mexican Restaurant,Hotel,Coffee Shop,Brewery,...,Museum,Music Venue,Performing Arts Venue,Gift Shop,Fast Food Restaurant,Pizza Place,Concert Hall,Art Gallery,Burger Joint,Southern / Soul Food Restaurant


In [62]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 2]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,"Albuquerque, N.M.",35.110830,-106.610000,2,Hotel,Mexican Restaurant,Furniture / Home Store,Burger Joint,Shoe Store,Brewery,...,Pizza Place,Rental Car Location,Thrift / Vintage Store,Gym,Park,Restaurant,Diner,Italian Restaurant,Video Store,Gas Station
4,"Atlanta, Ga.",33.755000,-84.390000,2,Hotel,American Restaurant,Aquarium,Coffee Shop,Hotel Bar,Sandwich Place,...,Burger Joint,Fast Food Restaurant,Seafood Restaurant,Caribbean Restaurant,Gym,Music Venue,Pizza Place,Park,Dive Bar,Burrito Place
6,"Baker, Ore.",44.775000,-117.834440,2,Coffee Shop,Hotel,Pizza Place,Fast Food Restaurant,Gas Station,Grocery Store,...,Diner,Sandwich Place,Tennis Court,Bowling Alley,Mexican Restaurant,Automotive Shop,Motel,Discount Store,Restaurant,Pharmacy
7,"Baltimore, Md.",39.283000,-76.617000,2,Hotel,American Restaurant,Aquarium,Seafood Restaurant,Bar,Sushi Restaurant,...,Dessert Shop,Gym,Burger Joint,Steakhouse,Mexican Restaurant,Middle Eastern Restaurant,Park,Peruvian Restaurant,Italian Restaurant,Coffee Shop
16,"Charleston, S.C.",32.783330,-79.933330,2,Hotel,Southern / Soul Food Restaurant,Coffee Shop,Seafood Restaurant,Restaurant,American Restaurant,...,Italian Restaurant,Clothing Store,Breakfast Spot,Sandwich Place,Pizza Place,Burger Joint,Cosmetics Shop,Theater,Ice Cream Shop,Cocktail Bar
18,"Charlotte, N.C.",35.227220,-80.843060,2,Pizza Place,Hotel,Steakhouse,American Restaurant,Theater,Restaurant,...,Southern / Soul Food Restaurant,Irish Pub,New American Restaurant,Lounge,French Restaurant,Coffee Shop,Tapas Restaurant,Creperie,Hotel Bar,Café
23,"Columbia, S.C.",34.000560,-81.034720,2,American Restaurant,Bar,Hotel,Coffee Shop,Seafood Restaurant,Café,...,Brewery,Pizza Place,Ice Cream Shop,Sandwich Place,Dessert Shop,Wine Bar,Burger Joint,Art Gallery,Steakhouse,Park
25,"Dallas, Tex.",32.779170,-96.808890,2,Hotel,Steakhouse,Bar,Coffee Shop,Park,Plaza,...,Italian Restaurant,Bakery,Seafood Restaurant,Taco Place,Art Museum,Cocktail Bar,New American Restaurant,Scenic Lookout,American Restaurant,Pizza Place
28,"Detroit, Mich.",42.331390,-83.045830,2,American Restaurant,Hotel,Coffee Shop,Steakhouse,Diner,Restaurant,...,Burger Joint,New American Restaurant,Cocktail Bar,Outdoor Sculpture,Jazz Club,Pub,Dessert Shop,Hot Dog Joint,Sandwich Place,Speakeasy
30,"Duluth, Minn.",46.786939,-92.098194,2,Hotel,Pizza Place,Brewery,Coffee Shop,American Restaurant,Trail,...,Bar,Movie Theater,Theater,Italian Restaurant,Candy Store,Café,Shopping Mall,Breakfast Spot,Speakeasy,Sporting Goods Shop


In [63]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 3]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
116,"Toledo, Ohio",41.66556,-83.57528,3,Discount Store,Intersection,Fast Food Restaurant,Art Museum,Park,Fried Chicken Joint,...,Storage Facility,Nightclub,Café,Bank,Skating Rink,Coffee Shop,Performing Arts Venue,Pharmacy,Disc Golf,Concert Hall


In [64]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 4]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Albany, N.Y.",42.652500,-73.757220,4,Pub,Café,Coffee Shop,Pizza Place,Theater,New American Restaurant,...,Pharmacy,Latin American Restaurant,Hotel,Mexican Restaurant,Asian Restaurant,Noodle House,Beer Bar,Steakhouse,Food & Drink Shop,Donut Shop
3,"Anchorage, Alaska",61.217000,-149.900000,4,Seafood Restaurant,Park,Coffee Shop,Clothing Store,Bar,Steakhouse,...,Restaurant,Asian Restaurant,Mexican Restaurant,Breakfast Spot,Pub,Department Store,Sushi Restaurant,Thai Restaurant,Bakery,Brewery
11,"Boise, Idaho",43.617000,-116.200000,4,Coffee Shop,Pizza Place,Hotel,Brewery,Farmers Market,Bar,...,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Pub,Deli / Bodega,Park,Indian Restaurant,Ice Cream Shop,Movie Theater,Chinese Restaurant
12,"Boston, Mass.",42.358060,-71.063610,4,Italian Restaurant,Park,Pizza Place,Bakery,Coffee Shop,Seafood Restaurant,...,Yoga Studio,Theater,Hotel,Salad Place,Gourmet Shop,Gym / Fitness Center,Gym,Steakhouse,Belgian Restaurant,Record Shop
14,"Calgary, Alba., Can.",51.050000,-114.067000,4,Coffee Shop,Steakhouse,Hotel,Restaurant,Pub,American Restaurant,...,Gastropub,Scenic Lookout,Chinese Restaurant,Italian Restaurant,French Restaurant,Cocktail Bar,Museum,Brazilian Restaurant,Performing Arts Venue,Pizza Place
27,"Des Moines, Iowa",41.590830,-93.620830,4,Coffee Shop,Italian Restaurant,Hotel,Bar,New American Restaurant,Café,...,Antique Shop,Pizza Place,Burger Joint,Furniture / Home Store,Brewery,Garden,Performing Arts Venue,Steakhouse,Latin American Restaurant,Bike Shop
32,"Edmonton, Alb., Can.",53.533000,-113.500000,4,Café,Bar,Coffee Shop,Italian Restaurant,Pizza Place,Pub,...,BBQ Joint,French Restaurant,Steakhouse,Breakfast Spot,Diner,Korean Restaurant,Park,Farmers Market,Chinese Restaurant,Indie Movie Theater
35,"Eugene, Ore.",44.051940,-123.086670,4,Brewery,Coffee Shop,Pizza Place,Thai Restaurant,Beer Store,Bar,...,Park,Sporting Goods Shop,Sushi Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Bakery,Arcade,American Restaurant,Performing Arts Venue,Sandwich Place
41,"Grand Rapids, Mich.",42.961110,-85.655560,4,Coffee Shop,American Restaurant,Brewery,Café,Hotel,Bar,...,Museum,Music Venue,Thai Restaurant,Sandwich Place,Italian Restaurant,Mexican Restaurant,Steakhouse,Art Gallery,Pub,Dessert Shop
44,"Honolulu, Hawaii",21.300000,-157.817000,4,Japanese Restaurant,Coffee Shop,Bakery,Dessert Shop,Chinese Restaurant,Café,...,Sushi Restaurant,Bar,Supermarket,Grocery Store,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Scenic Lookout,Pharmacy,Bubble Tea Shop


In [65]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 5]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
31,"Eastport, Maine",44.91361,-67.00389,5,Seafood Restaurant,Food,State / Provincial Park,Bakery,Gift Shop,Campground,...,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Zoo,Dumpling Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [66]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 6]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
53,"Key West, Fla.",24.559720,-81.783610,6,Hotel,Cuban Restaurant,Resort,Bed & Breakfast,Seafood Restaurant,American Restaurant,...,Garden,Vegetarian / Vegan Restaurant,Bar,Italian Restaurant,Café,Dive Bar,Diner,Dessert Shop,French Restaurant,Breakfast Spot
71,"Montreal, Que., Can.",45.508890,-73.561670,6,Café,French Restaurant,Hotel,Restaurant,Plaza,Cocktail Bar,...,Steakhouse,Art Museum,Coffee Shop,Taco Place,Breakfast Spot,Seafood Restaurant,Comedy Club,Tea Room,Italian Restaurant,Performing Arts Venue
78,"New York, N.Y.",40.661000,-73.944000,6,Caribbean Restaurant,Café,Bakery,Cocktail Bar,Juice Bar,Wine Shop,...,Bar,Bagel Shop,Grocery Store,Gym,Vegetarian / Vegan Restaurant,Deli / Bodega,Sushi Restaurant,Playground,Museum,Candy Store
104,"San Juan, P.R.",18.451522,-66.069481,6,Caribbean Restaurant,Hotel,Italian Restaurant,Restaurant,French Restaurant,Cocktail Bar,...,Beach,Thai Restaurant,Bar,Gym,Spanish Restaurant,Vegetarian / Vegan Restaurant,Cuban Restaurant,Pizza Place,Plaza,Sandwich Place


In [67]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 7]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,"Amarillo, Tex.",35.19917,-101.845280,7,Sandwich Place,Restaurant,American Restaurant,Bank,Gas Station,Bar,...,Burrito Place,Bakery,Baseball Stadium,Grocery Store,Gift Shop,Gay Bar,Theater,Hotel,Liquor Store,Coffee Shop
8,"Bangor, Maine",44.80000,-68.800000,7,Rental Car Location,Pizza Place,Hotel,Airport Terminal,American Restaurant,Coffee Shop,...,Gas Station,Pharmacy,Park,Golf Course,Baseball Stadium,Grocery Store,Soccer Field,Ice Cream Shop,Sushi Restaurant,Gift Shop
10,"Bismarck, N.D.",46.81333,-100.778890,7,Clothing Store,Pizza Place,Coffee Shop,Sandwich Place,American Restaurant,Fast Food Restaurant,...,Gift Shop,Lingerie Store,Mobile Phone Shop,Café,Burger Joint,Pub,Video Store,Cosmetics Shop,Discount Store,Diner
13,"Buffalo, N.Y.",42.90472,-78.849440,7,Discount Store,Bar,Intersection,Gay Bar,Pizza Place,Coffee Shop,...,Pharmacy,Cosmetics Shop,Car Wash,Science Museum,Snack Place,Rental Service,Recording Studio,Pub,Bus Stop,Convenience Store
17,"Charleston, W. Va.",38.34722,-81.633330,7,Pizza Place,Bar,Discount Store,American Restaurant,Bakery,New American Restaurant,...,Mexican Restaurant,Comfort Food Restaurant,Clothing Store,Lingerie Store,Sandwich Place,Convenience Store,Cosmetics Shop,Café,Seafood Restaurant,Indian Restaurant
26,"Denver, Colo.",39.76185,-104.881105,7,Coffee Shop,Pool,American Restaurant,Sandwich Place,Park,Breakfast Spot,...,Convenience Store,Mobile Phone Shop,Salon / Barbershop,Bagel Shop,Shopping Plaza,Gym / Fitness Center,Gas Station,Clothing Store,Video Store,Italian Restaurant
29,"Dubuque, Iowa",42.50000,-90.690000,7,Pizza Place,Bar,Mexican Restaurant,Coffee Shop,Golf Course,Italian Restaurant,...,Chinese Restaurant,Brewery,Burger Joint,Business Service,Fast Food Restaurant,Farmers Market,Nightclub,ATM,Light Rail Station,Salon / Barbershop
62,"Louisville, Ky.",38.22533,-85.741700,7,Bar,Pizza Place,Coffee Shop,Sandwich Place,Gas Station,Park,...,Bakery,Italian Restaurant,Bank,Gastropub,Restaurant,Dive Bar,Food Truck,Burger Joint,Mexican Restaurant,History Museum
63,"Manchester, N.H.",42.99083,-71.463610,7,Café,American Restaurant,Pizza Place,Donut Shop,Mediterranean Restaurant,Greek Restaurant,...,Mexican Restaurant,Diner,Chinese Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Gastropub,Deli / Bodega
64,"Memphis, Tenn.",35.11750,-89.971110,7,Discount Store,Bar,Café,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,...,Pizza Place,Hobby Shop,Seafood Restaurant,Music Store,Museum,Dog Run,Martial Arts Dojo,Gas Station,Record Shop,Donut Shop


In [68]:
uscan_merged.loc[uscan_merged['ClusterLabels'] == 8]

,City,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
93,"Richfield, Utah",38.76583,-112.0875,8,Pizza Place,Steakhouse,Fast Food Restaurant,Sandwich Place,Big Box Store,Taco Place,...,Hotel,Gas Station,Grocery Store,Ethiopian Restaurant,English Restaurant,Empanada Restaurant,Event Space,Electronics Store,Exhibit,Zoo
